In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob, os, ntpath

# Load data

In [ ]:
imgs_names = glob.glob('**/*.tif')
imgs = [cv.imread(name,0) for name in imgs_names]
imgs_names = [extract_name(name) for name in imgs_names] # leave only the file name so it's easy to add notation of augmentation

# Filter noise and detect holes

# Data augmentation

In [ ]:
def extract_name(path):
    head, tail = ntpath.split(path)
    head = tail or ntpath.basename(head)
    return os.path.splitext(head)[0]

In [ ]:
# labels = []
n = len(imgs)

# add noisy versions
noiseLvls = [0.2,0.1,0.05]
for i in range(n):
    row,col = imgs[i].shape
    name = imgs_names[i]
    for j,noise in enumerate(noiseLvls):
        imgs.append(imgs[i]+np.random.normal(0.0,noise,(row,col))*255)
        imgs_names.append(name+'_gn'+str(j))
        # labels.append(y)
    imgs.append(imgs[i]*(np.random.randn(row,col)*0.4+1))
    imgs_names.append(name+'_sn')
    # labels.append(y)

n = len(imgs)
rotations = [cv.ROTATE_90_CLOCKWISE, cv.ROTATE_180, cv.ROTATE_90_COUNTERCLOCKWISE]
# add rotated and flipped versions
for i in range(n):
    name = imgs_names[i]
    # y = labels[i]
    # add rotated versions
    imgs.append(cv.flip(imgs[i],1))
    imgs_names.append(name+'_fr000')
    # labels.append(cv.flip(y,1))
    for j,r in enumerate(rotations):
        angle = str((j+1)*90)
        imgs.append(cv.rotate(imgs[i],r))
        imgs_names.append(name+'_r'+angle)
        # labels.append(cv.rotate(y,r))
        imgs.append(cv.rotate(cv.flip(imgs[i],1),r))
        imgs_names.append(name+'_fr'+angle)
        # labels.append(cv.rotate(cv.flip(y,1),r))

In [ ]:
n = 25
a = np.ceil(np.sqrt(len(imgs[:n])))
plt.figure(figsize=(40,40))
for i in range(len(imgs[:n])):
    plt.subplot(a,a,i+1)
    plt.imshow(imgs[i])
    plt.title(imgs_names[i])
plt.show()